In [ ]:
!nvidia-smi

In [1]:
# !pip install -q keras
# !pip install -q pandas
# !pip install -q numpy
# !pip install -q tensorflow
# !pip install -U -q PyDrive

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import os
# os.chdir("drive/My Drive/colab_datas/teknofest/keras-retinanet")

In [ ]:
!pip install -q keras_resnet

In [3]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import numpy as np
import time
import glob
import pandas as pd
# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm

def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

Using TensorFlow backend.


## Video Pred

In [ ]:
input_video="drone_1"

vidcap = cv2.VideoCapture('drone_night_footage/{}.mp4'.format(input_video))
success,image = vidcap.read()
count = 0
frames=[]
while success:
    frames.append(image)     # save frame as JPEG file      
    success,image = vidcap.read()
    # print('Read a new frame: ', success)
    count += 1

In [ ]:
frames=glob.glob("photos/test_images/*")

In [ ]:
labels_to_names = {0: 'yaya', 1: 'arac'}
!python keras_retinanet/bin/convert_model.py ./snapshots_woconfig_waug/resnet50_csv_11.h5 ./snapshots/infer_model.h5
model = models.load_model("./snapshots/infer_model.h5", backbone_name='resnet50')
resnet50_images=[]
for image in tqdm(frames):# load image
    sha=image.shape
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    image = preprocess_image(image)
    image, scale = resize_image(image,max_side=3000)
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))

    boxes /= scale

    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < 0.5:
            break
        color = label_color(label)
        b = box.astype(int)
        draw_box(draw, b, color=color)
        
        caption = "{} {:.3f}".format(labels_to_names[label], score)
        draw_caption(draw, b, caption)
    resnet50_images.append(draw)
# os.mkdir("snapshots_resnet50_csv_01")
# for i in range(len(resnet50_images)):
#     image_=cv2.cvtColor(resnet50_images[i],cv2.COLOR_BGR2RGB)
#     cv2.imwrite("snapshots_resnet50_csv_01/{}.jpg".format(i),image_)
#     # plt.figure(figsize=(20, 20))
#     # plt.axis('off')
#     # plt.imshow(image)
#     # plt.show()
#     # plt.pause(0.01)

In [ ]:
from os.path import isfile, join
height, width, layers = resnet50_images[0].shape

out = cv2.VideoWriter("drone_night_footage/{}_pred_lastest_waug.avi".format(input_video),cv2.VideoWriter_fourcc(*"MJPG"), 60, (width,height))
for i in tqdm(range(len(resnet50_images))):
    out.write(resnet50_images[i])
    # cv2.imwrite("drone_night_footage/"+str(i)+".jpg",resnet50_images[i])
cv2.destroyAllWindows()
out.release()

In [ ]:
# [os.remove(i) for i in glob.glob("drone_night_footage/*.jpg")]

## Modellerin birleşmesi

In [ ]:
paths=glob.glob("photos/test_images/*")

In [ ]:
# 4 farklı modelin karsılastırılması 

model_paths=["./snapshots/son_50.h5","./snapshots/resnet101_csv_06.h5","./snapshots_woconfig/resnet50_csv_14.h5","./snapshots_woconfig_waug/resnet50_csv_11.h5"]
labels_to_names = {0: 'yaya', 1: 'arac'}
pred_dict={}
for model_path in model_paths:
    !python keras_retinanet/bin/convert_model.py $model_path ./snapshots/infer_model.h5
    if "resnet101" in model_path:
        model = models.load_model("./snapshots/infer_model.h5", backbone_name='resnet101')
    else:
        model = models.load_model("./snapshots/infer_model.h5", backbone_name='resnet50')
    pred_dict[model_path]=[]
    print(model_path)
    for path in tqdm(paths):
        image=read_image_bgr(path)
        sha=image.shape
        # draw = image.copy()
        # draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
        image = preprocess_image(image)
        image, scale = resize_image(image,max_side=3000)
        boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
        boxes /= scale
        bbs=[]
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            if score < 0.5:
                break
            # color = label_color(label)
            b = box.astype(int)
            bbs.append([path,b[0],b[1],b[2],b[3],labels_to_names[label], score])
            # draw_box(draw, b, color=color)
            # caption = "{} {:.3f}".format(labels_to_names[label], score)
            # draw_caption(draw, b, caption)
        # font = cv2.FONT_HERSHEY_SIMPLEX
        # cv2.putText(draw,model_path.split("/")[-1].split(".")[0],(10,500), font, 4,(255,255,255),2,cv2.LINE_AA)
        pred_dict[model_path].append(bbs)

In [ ]:
pred_dict.keys()

In [ ]:
# pred_dict["./snapshots/son_50.h5"]=[i for i in pred_dict["./snapshots/son_50.h5"]  if not len(i)==0]
# pred_dict["./snapshots/resnet101_csv_06.h5"]=[i for i in pred_dict["./snapshots/resnet101_csv_06.h5"]  if not len(i)==0]
# pred_dict["./snapshots_woconfig_waug/resnet50_csv_11.h5"]=[i for i in pred_dict["./snapshots_woconfig_waug/resnet50_csv_11.h5"]  if not len(i)==0]
# pred_dict["./snapshots_woconfig/resnet50_csv_14.h5"]=[i for i in pred_dict["./snapshots_woconfig/resnet50_csv_14.h5"]  if not len(i)==0]

In [ ]:
df=[]
for key in pred_dict.keys():
  for frame in pred_dict[key]:
    for pred in frame[:-1]:
      pred.append(key)
      df.append(pred)

In [ ]:
df=pd.DataFrame(df)
df

In [ ]:
df.to_csv("pred_model_comparison.csv",index=False)

In [ ]:
df.head()

In [ ]:
df[7].value_counts().index

In [4]:
labels_to_names_rev={'yaya':0,  'arac':1}

In [ ]:
frame_paths=pd.DataFrame(df[0].value_counts().index)
frame_paths[1]=frame_paths[0].apply(lambda x:x.split("/")[-1].split(".")[0]).astype(int)
frame_paths=frame_paths.sort_values(by=[1])
frame_paths

In [ ]:
from os.path import isfile, join
# height, width, layers = image.shape

out = cv2.VideoWriter("project_11.avi",cv2.VideoWriter_fourcc(*"DIVX"), 7.5, (1920,1080))

for frame in tqdm(frame_paths[0].values):
    image=cv2.imread(frame)
    images_dict={}
    models=df[7].value_counts().index
    for model_path in models:
        image_1=image.copy()
        unq_df=df.loc[(df[7]==model_path)&(df[0]==frame)]
        for bb in unq_df.values:
            color = label_color(labels_to_names_rev[bb[5]])
            draw_box(image_1, bb[1:5], color=color)
            caption = "{} {:.3f}".format(bb[5], bb[6])
            draw_caption(image_1, bb[1:5], caption)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(image_1,model_path.split("/")[-1].split(".")[0],(10,500), font, 4,(255,255,255),2,cv2.LINE_AA)
        images_dict[model_path]=image_1.copy()
    image_1=np.concatenate((images_dict["./snapshots/son_50.h5"],images_dict["./snapshots_woconfig_waug/resnet50_csv_11.h5"]),axis=0)
    image_2=np.concatenate((images_dict["./snapshots_woconfig/resnet50_csv_14.h5"],images_dict["./snapshots/resnet101_csv_06.h5"]),axis=0)
    image_1=np.concatenate((image_1,image_2),axis=1)
    image_1=cv2.resize(image_1,(1920,1080))
    out.write(image_1)
cv2.destroyAllWindows()
out.release()

## Video cıkar



In [5]:
paths=glob.glob("photos/test_images/*")

In [9]:
paths=[i.replace("\\","/") for i in paths]

In [14]:
model_preds=pd.read_csv("resnet50_preds_scaled.csv")
model_preds["path"]=model_preds["path"].apply(lambda x: x.replace("\\","/"))
model_preds

In [17]:
from os.path import isfile, join
# height, width, layers = image.shape
out = cv2.VideoWriter("project_1222.avi",cv2.VideoWriter_fourcc(*"DIVX"), 7.5, (960,540))

for frame in tqdm(paths):
    image=cv2.imread(frame)
    images_dict={}
    image_1=image.copy()
    image_1=cv2.cvtColor(image_1,cv2.COLOR_BGR2RGB)
    unq_df=model_preds[model_preds["path"]==frame]
    for bb in unq_df.values:
        color = label_color(labels_to_names_rev[bb[5]])
        draw_box(image_1, bb[1:5], color=color)
        caption = "{} {:.3f}".format(bb[5], bb[6])
        draw_caption(image_1, bb[1:5], caption)
    image_1=cv2.resize(image_1,(960,540))
    out.write(image_1)
cv2.destroyAllWindows()
out.release()

100%|██████████████████████████████████████████████████████████████████████████████| 1886/1886 [01:14<00:00, 25.15it/s]


## Blended predictions

In [ ]:
preds=pd.read_csv("./pred_model_comparison.csv")
preds.columns=["path","x1","y1","x2","y2","label","confidence","model"]
preds["area"]=(preds["x2"]-preds["x1"])*(preds["y2"]-preds["y1"])
preds["center_x"]=((preds["x2"].astype(int)+preds["x1"].astype(int))/2).astype(int)
preds["center_y"]=((preds["y2"].astype(int)+preds["y1"].astype(int))/2).astype(int)
preds=preds.sort_values(by=["path","center_x","center_y"])
preds.head()

In [ ]:
preds=preds.drop(preds.loc[preds["model"]=="./snapshots/resnet101_csv_06.h5"].index)
preds

In [ ]:
frame_paths=pd.DataFrame(preds["path"].value_counts().index)
frame_paths[1]=frame_paths[0].apply(lambda x:x.split("/")[-1].split(".")[0]).astype(int)
frame_paths=frame_paths.sort_values(by=[1])
frame_paths.head()

In [ ]:
for i, row in preds.iterrows():
    break
i,row

In [ ]:
# buraya iki data data frame i alarak ensemble i koy

def find_intersection(df1,df2):
    new_preds=[]
    idx_tbd_1=[]
    idx_tbd_2=[]
    for idx_df1,bb_df1 in df1.iterrows():
        for idx_df2,bb_df2 in df2.iterrows():
            if abs(bb_df1["center_x"]-bb_df2["center_x"])<=20 and abs(bb_df1["center_y"]-bb_df2["center_y"])<=20:
                frame=bb_df1["path"]
                x1=min(bb_df1["x1"],bb_df2["x1"])
                y1=min(bb_df1["y1"],bb_df2["y1"])
                x2=min(bb_df1["x2"],bb_df2["x2"])
                y2=min(bb_df1["y2"],bb_df2["y2"])
                label_=bb_df1["label"]
                conf=bb_df1["confidence"]
                idx_tbd_1.append(idx_df1)
                idx_tbd_2.append(idx_df2)
                new_preds.append([frame,x1,y1,x2,y2,label_,conf,"ens",(x2-x1)*(y2-y1),bb_df1["center_x"],bb_df1["center_y"]])
    df1=df1.drop(idx_tbd_1)
    df2=df2.drop(idx_tbd_2)
    new_preds=pd.DataFrame(new_preds)
    try:
        new_preds.columns=df1.columns
    except:
        pass
    return df1,df2,new_preds

In [ ]:
indexes_tbd=[]
model_preds=[]
for frame in tqdm(frame_paths[0].values):
    unq_df=preds.loc[preds["path"]==frame]
    son_50_model=unq_df.loc[unq_df["model"]=="./snapshots/son_50.h5"]
    confident_model=unq_df.loc[unq_df["model"]=="./snapshots_woconfig/resnet50_csv_14.h5"]
    augmented_model=unq_df.loc[unq_df["model"]=="./snapshots_woconfig_waug/resnet50_csv_11.h5"]
    son_1,conf_1,ens_1=find_intersection(son_50_model,confident_model)
    ens_2,aug_1,ens_3=find_intersection(ens_1,augmented_model)
    for i in son_1.values:
        if not len(i)==0:
            model_preds.append(i)
    for i in conf_1.values:
        if not len(i)==0:
            model_preds.append(i)
    for i in ens_2.values:
        if not len(i)==0:
            model_preds.append(i)
    for i in aug_1.values:
        if not len(i)==0:
            model_preds.append(i)
    for i in ens_3.values:
        if not len(i)==0:
            model_preds.append(i)

In [ ]:
model_preds

In [ ]:
model_preds=pd.DataFrame(model_preds)

In [ ]:
model_preds.to_csv("preds_tuned.csv",index=False,header=None)